## SICP 习题 （3.5）解题总结: 蒙特卡洛模拟

SICP 习题2.5 引入了 蒙特卡洛模拟 的概念，虽然书中这段内容还是关于赋值的讨论，但是它引入了一个非常有深度的概念，就是这个 蒙特卡洛模拟。

对蒙特卡洛模拟大家可能都感觉比较陌生，但是相信大家都知道AlphaGo。 在AlphaGo的算法实现里，蒙特卡洛模拟是非常关键的，具体讲是一个叫蒙特卡洛树搜索的一种技术。

我当时看这道题的时候也是跳过了的，一方面是因为没有太深入理解蒙特卡洛模拟的概念，另一方面是因为Scheme里没有实现rand函数来生成随机数，写的代码都无法测试。

后来因为要去实现AlphaGo论文的具体算法，自己重新学习了一下蒙特卡洛模拟的思路，才发现蒙特卡洛模拟这么强大。再者，因为Calysto Scheme可以导入python模块，那就可以在Scheme代码里实现rand函数了。

所以有了下面的解题总结。

### 蒙特卡洛模拟

先看看蒙特卡洛模拟，它的基本思路就是通过随机模拟去逼近一个希望计算的值。

听起来比较抽象，先举个简单的例子。比如你想算出来1/3的数值是多少，用蒙特卡洛模拟的思路可以这么做：

1. 划定一个田字格，分别标上1，2，3，4的标号，这样你就有了四个相同的方格。

2. 往田字格上撒盐，你没看错，就是撒盐，像放调料一样，不过要撒均匀一点

3. 收集第一格上的盐粒，称个重

4. 再收集 2， 3， 4格上的盐粒，将这三个格子的盐也称个重。

5. 把第3步的称重结果除于第4步的称重结果就会约等于1/3

这个事情挺直观的，如果我们撒盐撒的比较均匀，一个格子的盐粒重量除于三个格子的盐粒重量不就是1/3嘛。

不过1/3的值我们都知道啦，这么辛苦各种操作计算个1/3有什么意义吗？

确实没啥意义，但是如果我告诉你类似的方法可以用来算 $\pi$你会觉得有点意思吗？

确实可以，这个也是SICP 题目 3.5 要我们做的事情，基本操作如下：

1. 在纸面上画个正方形
2. 在这个正方形里画个内切圆，如果内切圆是啥有点不记得了就直接看下面的图
3. 往纸上均匀地撒盐
4. 收集正方形内，但是在圆以外的盐粒。称重
5. 收集圆内的盐粒。称重
6. 把第4步，第5步的结果加起来，就是正方形内所有盐粒的重量
7. 把圆内的盐粒重量除于正方形内所有盐粒的重量，猜猜结果是什么


示意图如下：![](./image/3-05.png)

结果会接近 $\frac {\pi} {4}$!

为啥？我们看看它们的面积关系，如果圆的半径是r，那么正方形的边长就是2r，圆的面积除于正方形的面积就是：$ \frac {\pi r^2} {(2r)^2} = \frac {\pi r^2} {4r^2} = \frac {\pi} {4}$

因为我们撒盐撒的比较均匀，所以盐粒的重量和面积是成正比的。

这就是蒙特卡洛模拟的核心思想，如果我们模拟的次数越多，那结果就越接近我们期望的结果。

### 代码实现

接着就是如何在代码里实现它了，基本思路和上面差不多，不过需要转换到直角坐标系上。

我们随机生成一个点（x,y）要求x和y都在[-1, 1]的范围内，用这一个随机的点模拟一个撒下去的盐粒，被撒在一个边长为2的正方形内。

然后，检查x和y, 如果 $ x^2 + y^2 <= 1$, 那么这个“盐粒”就在半径为1的圆内

重复执行10000次，看看有多少个“盐粒”在圆内，这样就可以模拟上面的撒盐的动作了，然后除一下我们应该可以得到 $\frac {\pi} {4}$

首先有个问题是怎么生成随机数？SICP书里有讨论随机数实现的困难，同时假设我们已经有了一个叫rand的函数。

不过事实上Scheme上没有实现rand，我们就只好自己打包一个python接口出来：

In [2]:
(import 'random)
(python-exec "
def rand():
    return random.random()
             ")

In [3]:
(rand)

0.26036061480085604

这个函数返回的是从0到1之间的一个随机小数，刚才我们讨论需要一个从-1到1的随机数，所以我们再打包一个：

In [11]:
(define (rand-between min max)
  (let ((scale (- max min)))
    (let((shift (/ scale 2)))
        (- (* (rand) scale) shift)
    )))

(define (rand-for-pie)
  (rand-between -1 1))

In [12]:
(rand-between -1 1)

-0.31862095480766817

这样我们就打造了一个专属的随机函数，每次调用都会返回一个-1到1之间的实数

In [46]:
(rand-for-pie)

0.519105695438598

接着定义我们的校验函数，在上文的讨论中我们是希望知道落在圆内的“盐粒”有多少，所以定义一个(area-test),如果$x^2 + y^2 < 1$就返回#t

In [47]:
(define (square x) (* x x))
(define (area-test)
  (let ((x (rand-for-pie))
        (y (rand-for-pie)))
    (< (+ (square x) (square y)) 1)
    ))

多跑几次，可以发现有时“盐粒”落在圆内，有时落在圆外：

In [51]:
(display (area-test))(newline)
(display (area-test))(newline)
(display (area-test))(newline)
(display (area-test))(newline)
(display (area-test))(newline)
(display (area-test))(newline)


True
True
False
True
True
True


然后定义蒙特卡洛的主体函数，就是调用指定次数的校验函数，返回的值是通过校验的数量除于整体数量：

In [52]:
(define (monte-carlo trials experiment)
  (define (iter trials-remaining trials-passed)
    (cond ((= trials-remaining 0)
	   (/ trials-passed trials))
	  ((experiment)
	   (iter (- trials-remaining 1) (+ trials-passed 1)))
	  (else 
	   (iter (- trials-remaining 1) trials-passed))))
  (iter trials 0))

调用以上方法，测试10000次，最后乘于4，因为我们的蒙特卡洛模拟算出的是$\frac {\pi} {4}$, 乘于4就是$\pi$了

In [53]:
(* (monte-carlo 10000 area-test) 4.0)

3.1452

算出来的结果大概是3.14，3.15之类的，如果模拟次数增加还可以更精确一些。

以上就是3.5题目的要求了，如果你还想测试一下书里正文提到的另一个计算$\pi$的思路，可以尝试下面的代码。

它的基本思路就是随机两个整数没有公因子的概率是 $ \frac {6} {\pi^2}$

首先我们要生成随机整数，我们新打包一个python接口：

In [60]:
(python-exec "
def randint():
    return random.randint(0,10000)
             ")

In [61]:
(randint)

3053

然后通过(gcd)计算两个整数有没有公因子：

In [57]:
(define (cesaro-test)
  (= (gcd (randint) (randint)) 1))

In [58]:
(define (gcd a b)
	(if (= b 0)
		a
		(gcd b (remainder a b))))

最后就是通过蒙特卡洛模拟计算$\pi$

In [59]:
(sqrt (/ 6.0 (monte-carlo 1000 cesaro-test)))

3.1916699728542883